In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, f1_score

from imblearn.over_sampling import SMOTE

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import os
import sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
from src.functions import load_OU_data, CourseScaler, plot_confusion, score_grid, \
smotecourses, process_courses, course_cross_validate, Course_GridSearchCV, \
model_evaluate_presentation, cross_val_presentation, GridSearchPresentationCV

%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np

import keras
import tensorflow as tf
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from keras.layers import Input, Dense, Dropout, Activation, LeakyReLU
from keras.models import Sequential
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.callbacks import ModelCheckpoint
from keras.callbacks import LearningRateScheduler

from sklearn.preprocessing import LabelEncoder

!pip install -q -U keras-tuner
import kerastuner as kt

import pickle

import warnings

from IPython.display import display

warnings.filterwarnings("ignore")
sns.set_style('white')
pd.set_option('display.max_columns',None)
tf.keras.backend.set_floatx('float32')

In [ ]:
def graph_model_history(history):
    """
    Takes as argument a model history: either the return of a KerasClassifier or the model.history of a Keras model.
    Plots model accuracy, validation accuracy, model loss, and validation loss training histories, if they are present in the history.history dictionary object:
    """
        
    import seaborn as sns
    import matplotlib.pyplot as plt

    fig, axes = plt.subplots(1,2, figsize = (10,5))
    if 'acc' in history.history.keys():
        axes[0].plot(history.history['acc'], label = 'Train Accuracy')
    if 'val_acc' in history.history.keys():
        axes[0].plot(history.history['val_acc'], label = 'Validation Accuracy')
    axes[0].legend()
    if 'loss' in history.history.keys():
        axes[1].plot(history.history['loss'], label = 'Train Loss')
    if 'val_loss' in history.history.keys():
        axes[1].plot(history.history['val_loss'], label = 'Validation Loss')
    axes[1].legend()
    plt.show()

In [ ]:
df_full = load_OU_data(prediction_window=.75)


In [ ]:
df = df_full.copy()
df = df.drop(columns = ['id_student','region','highest_education', \
                   'imd_band','gender','age_band','disability','studied_credits',
                   'module_presentation_length','date_registration'])
df['final_result'] = np.array([0 if w in ['Pass','Distinction'] \
              else 1 for w in df['final_result']])
test_df = df[df['code_presentation'] == '2014J']

test_df = test_df[test_df['code_module'] != 'BBB']
train_df = df[df['code_presentation'] != '2014J']

X_train = train_df.drop(columns = ['final_result','code_presentation'])
y_train = train_df['final_result']
X_test = test_df.drop(columns = ['final_result','code_presentation'])
y_test = test_df['final_result']


X_train_transformed, y_train_transformed, X_test_transformed \
                                = process_courses(X_train, y_train, X_test)

In [ ]:
y_test.value_counts()

In [ ]:
def make_model1():
    model = Sequential()
    model.add(Dense(5, activation='relu'))

    model.add(Dense(2, activation='sigmoid'))

    model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['acc'])

    return model

In [ ]:
model = make_model1()
history = model.fit(X_train_transformed, y_train_transformed,
          epochs = 25,
          validation_data = (X_test_transformed, y_test))
y_pred = np.argmax(model.predict(X_test_transformed), axis=1)
print(accuracy_score(y_test, y_pred))
graph_model_history(history)
# model.save('densemodel1.h5')

In [ ]:
model.save('/content/densefsm.h5')

In [ ]:
def make_model2():
    model = Sequential()
    model.add(Dense(5, activation='relu'))
    model.add(Dense(5, activation='relu'))
    model.add(Dense(5, activation='relu'))

    model.add(Dense(2, activation='sigmoid'))

    model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['acc'])

    return model

model = make_model2()
history = model.fit(X_train_transformed, y_train_transformed,
          epochs = 5,
          validation_data = (X_test_transformed,y_test))
y_pred = np.argmax(model.predict(X_test_transformed), axis=1)
print(accuracy_score(y_test, y_pred))
graph_model_history(history)
model.save('/content/densemodel2.h5')

In [ ]:
def make_model3():
    model = Sequential()
    model.add(Dense(5, activation='relu'))
    model.add(Dropout(rate = .2))
    model.add(Dense(5, activation='relu'))
    model.add(Dropout(rate = .2))    
    model.add(Dense(5, activation='relu'))
    model.add(Dropout(rate = .2))

    model.add(Dense(2, activation='sigmoid'))


    model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['acc'])

    return model

model = make_model3()
history = model.fit(X_train_transformed, y_train_transformed,
          epochs = 5,
          validation_data = (X_test_transformed,y_test))
y_pred = np.argmax(model.predict(X_test_transformed), axis=1)
print(accuracy_score(y_test, y_pred))
graph_model_history(history)
model.save('/content/densemodel3.h5')

In [ ]:
def make_model4():
    model = Sequential()
    model.add(Dense(25, activation='relu'))
    model.add(Dropout(rate=.2))
    model.add(Dense(25, activation='relu'))
    model.add(Dropout(rate=.2))
    model.add(Dense(25, activation='relu'))
    model.add(Dropout(rate=.2))

    model.add(Dense(2, activation='sigmoid'))

    lr_schedule = keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=1e-2,
        decay_steps=1000,
        decay_rate=0.9)
    optimizer = keras.optimizers.SGD(learning_rate=lr_schedule)    
    model.compile(optimizer = optimizer, loss = 'sparse_categorical_crossentropy', metrics = ['acc'])

    return model

model = make_model4()
history = model.fit(X_train_transformed, y_train_transformed,
          epochs = 25,
          validation_data = (X_test_transformed,y_test))
y_pred = np.argmax(model.predict(X_test_transformed), axis=1)
print(accuracy_score(y_test, y_pred))
graph_model_history(history)
model.save('/content/densemodel4.h5')

In [ ]:
def make_model5():
    model = Sequential()
    model.add(Dense(25, activation='relu'))
    model.add(Dropout(rate=.2))
    model.add(Dense(25, activation='relu'))
    model.add(Dropout(rate=.2))
    model.add(Dense(25, activation='relu'))
    model.add(Dropout(rate=.2))
    model.add(Dense(25, activation='relu'))
    model.add(Dropout(rate=.2))
    model.add(Dense(2, activation='sigmoid'))

    lr_schedule = keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=1e-2,
        decay_steps=1000,
        decay_rate=.9)
    optimizer = keras.optimizers.Adam(learning_rate=lr_schedule)    
    model.compile(optimizer = optimizer, loss = 'sparse_categorical_crossentropy', metrics = ['acc'])

    return model

model = make_model5()
history = model.fit(X_train_transformed, y_train_transformed,
          epochs = 10,
          validation_data = (X_test_transformed,y_test))
y_pred = np.argmax(model.predict(X_test_transformed), axis=1)
print(accuracy_score(y_test, y_pred))
graph_model_history(history)
model.save('/content/densemodel5.h5')

In [ ]:
def make_model6():
    model = Sequential()
    model.add(Dense(50, activation='relu'))
    model.add(Dropout(rate=.2))
    model.add(Dense(50, activation='relu'))
    model.add(Dropout(rate=.2))
    model.add(Dense(25, activation='relu'))
    model.add(Dropout(rate=.2))
    model.add(Dense(10, activation='relu'))
    model.add(Dropout(rate=.2))
    model.add(Dense(50, activation='relu'))
    model.add(Dropout(rate=.2))
    model.add(Dense(50, activation='relu'))
    model.add(Dropout(rate=.2))
    model.add(Dense(25, activation='relu'))
    model.add(Dropout(rate=.2))
    model.add(Dense(10, activation='relu'))
    model.add(Dropout(rate=.2))
    model.add(Dense(2, activation='sigmoid'))

    lr_schedule = keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=1e-2,
        decay_steps=10000,
        decay_rate=.9)
    optimizer = keras.optimizers.SGD(learning_rate=lr_schedule)    
    model.compile(optimizer = optimizer, loss = 'sparse_categorical_crossentropy', metrics = ['acc'])

    return model

model = make_model6()
history = model.fit(X_train_transformed, y_train_transformed,
          epochs = 500,
          validation_data = (X_test_transformed,y_test),
          batch_size = 200)
y_pred = np.argmax(model.predict(X_test_transformed), axis=1)
print(accuracy_score(y_test, y_pred))
graph_model_history(history)
# model.save('/content/densemodel6.h5')

In [ ]:
def make_model7():
    model = Sequential()
    model.add(Dense(500, activation='relu'))
    model.add(Dropout(rate=.5))
    model.add(Dense(500, activation='relu'))
    model.add(Dropout(rate=.5))
    model.add(Dense(250, activation='relu'))
    model.add(Dropout(rate=.5))
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(rate=.2))
    model.add(Dense(50, activation='relu'))
    model.add(Dropout(rate=.2))
    model.add(Dense(50, activation='relu'))
    model.add(Dropout(rate=.2))
    model.add(Dense(25, activation='relu'))
    model.add(Dropout(rate=.2))
    model.add(Dense(10, activation='relu'))
    model.add(Dropout(rate=.2))
    model.add(Dense(2, activation='sigmoid'))

    lr_schedule = keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=1e-2,
        decay_steps=1000,
        decay_rate=.5)
    optimizer = keras.optimizers.Adam(
        # learning_rate=lr_schedule
        )    
    model.compile(optimizer = optimizer, loss = 'sparse_categorical_crossentropy', metrics = ['acc'])

    return model

model = make_model7()
history = model.fit(X_train_transformed, y_train_transformed,
          epochs = 10,
          validation_data = (X_test_transformed,y_test),
          batch_size = 50)
y_pred = np.argmax(model.predict(X_test_transformed), axis=1)
print(accuracy_score(y_test, y_pred))
graph_model_history(history)
model.save('/content/densemodel7.h5')

In [ ]:
def make_model7():
    model = Sequential()
    model.add(Dense(10, activation='tanh'))
    model.add(Dropout(rate=.1))
    model.add(Dense(2, activation='sigmoid'))

    lr_schedule = keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=1e-2,
        decay_steps=1000,
        decay_rate=.9)
    optimizer = keras.optimizers.RMSprop(
        learning_rate= lr_schedule
        # beta_1 = .9,
        # beta_2 = .1
        )    
    model.compile(optimizer = optimizer, loss = 'sparse_categorical_crossentropy', metrics = ['acc'])

    return model

model = make_model7()
history = model.fit(X_train_transformed, y_train_transformed,
          epochs = 5,
          validation_data = (X_test_transformed,y_test),
          batch_size = 100)
y_pred = np.argmax(model.predict(X_test_transformed), axis=1)
print(accuracy_score(y_test, y_pred))
graph_model_history(history)
model.save('/content/densemodel7.h5')

In [ ]:
eval_df = test_df
eval_df['prediction'] = y_pred
modules = eval_df['code_module'].unique()
course_acc = pd.DataFrame(columns = ['score'],index = modules)
fig, axes = plt.subplots(3,2, figsize = (10,10))
axes = axes.ravel()
for i, module in enumerate(modules):
    preds = eval_df[eval_df['code_module'] == module]['prediction']
    trues = eval_df[eval_df['code_module'] == module]['final_result']
    accuracy = accuracy_score(trues,preds)
    course_acc.loc[module,'score'] = accuracy
    plot_confusion(trues,preds, ax = axes[i])
    axes[i].set_title(f'module: {module}: accuracy score: {round(accuracy,2)}')
plt.tight_layout()
plt.show()